In [1]:
import cv2
#Cargar modelos 
faceClassifier = cv2.CascadeClassifier('content/haarcascade_frontalface_default.xml')#el archivo xml tiene el modelo ya entrenado
eyeClassifier = cv2.CascadeClassifier('content/haarcascade_eye.xml')
smileClassifier = cv2.CascadeClassifier('content/haarcascade_smile.xml')
#Iniciar la camara
camera = cv2.VideoCapture(0)

#Inicia un ciclo while hassta que la cama web deje de funcionar
while camera.isOpened():
    #lee la imagen capturada por la camara
    ok, frame = camera.read()
    if not ok:
        break
    #Convierte la imagen a formato RGB y luego la pasa a escala de grises
    rgbFrame = frame[:,:,::-1]
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #rectangulos de los restros detectados 
    faces = faceClassifier.detectMultiScale(grayFrame, scaleFactor=1.3, minNeighbors=6, minSize=(30,30),)
    for(x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.putText(frame,'face',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2 )
        #pasa a escala de grises el area del rostro detectado
        faceDetectedGray = grayFrame[y:y+h, x:x+w]
        #conserva el area del rostro detectado en color
        faceDetectedColor = frame[y:y+h, x:x+w]
        #deteccion de los ojos 
        eyes = eyeClassifier.detectMultiScale(faceDetectedGray, scaleFactor=1.1, minNeighbors=15, minSize=(2,2),)
        for(ex,ey,ew,eh) in eyes:
            cv2.rectangle(faceDetectedColor,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            cv2.putText(frame,'eye',(x+ex+10,y+ey+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            #deteccion de la sonrisa
        smiles = smileClassifier.detectMultiScale(faceDetectedGray, scaleFactor=1.2,minNeighbors=22, minSize=(2,2),)
        for(sx,sy,sw,sh) in smiles:
            cv2.rectangle(faceDetectedColor,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)
            cv2.putText(frame,'smile',(x+sx+10,y+sy+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
    #Traza la imagen de prueba con los respectivos rectangulos limitantes
    cv2.imshow('Face, Eye and Smile Detection', frame)
    key=cv2.waitKey(30)
    if key==27:#si se presiona la tecla esc se sale del ciclo
        break
#Libera la camara y destruye todas las ventanas
camera.release()
cv2.destroyAllWindows()